파일병합

In [ ]:
# -*- coding: utf-8 -*-
"""
Google Colab에서 여러 개의 파일을 병합하는 스크립트입니다.
이 코드는 모든 파일의 열(column)이 동일하다고 가정합니다.
CSV와 Excel(xlsx) 파일을 지원합니다.
"""

import pandas as pd
from google.colab import files
import io

# 1. 파일 업로드
# 이 셀을 실행하면 파일 선택창이 나타납니다.
# 병합하려는 파일 3개를 모두 선택하여 업로드해주세요.
print("병합할 파일 3개를 업로드해주세요. (CSV 또는 XLSX)")
uploaded = files.upload()

# 업로드된 파일이 3개가 아니면 오류 메시지 출력
if len(uploaded) != 3:
    print(f"\n[오류] 정확히 3개의 파일을 업로드해야 합니다. (업로드된 파일 수: {len(uploaded)}개)")
else:
    # 데이터프레임을 저장할 리스트 생성
    df_list = []

    # 2. 업로드된 각 파일을 DataFrame으로 읽기
    for filename, content in uploaded.items():
        print(f"'{filename}' 파일을 읽는 중...")
        try:
            # 파일 확장자에 따라 적절한 함수 사용
            if filename.lower().endswith('.csv'):
                # io.BytesIO를 사용하여 업로드된 파일 내용을 읽습니다.
                # 다양한 인코딩 문제를 방지하기 위해 'utf-8-sig' 또는 'cp949' 시도
                try:
                    df = pd.read_csv(io.BytesIO(content))
                except UnicodeDecodeError:
                    df = pd.read_csv(io.BytesIO(content), encoding='cp949')
                df_list.append(df)
            elif filename.lower().endswith('.xlsx'):
                df = pd.read_excel(io.BytesIO(content))
                df_list.append(df)
            else:
                print(f"'{filename}'은(는) 지원하지 않는 파일 형식입니다.")
        except Exception as e:
            print(f"'{filename}' 파일을 읽는 중 오류가 발생했습니다: {e}")

    # 3. 모든 DataFrame을 하나로 병합
    if len(df_list) == 3:
        # ignore_index=True는 기존 인덱스를 무시하고 새로운 인덱스를 부여합니다.
        merged_df = pd.concat(df_list, ignore_index=True)

        # 4. 결과 확인
        print("\n 파일 병합이 성공적으로 완료되었습니다.")
        print("\n[병합된 데이터 정보]")
        merged_df.info()

        print("\n[병합된 데이터 미리보기 (상위 5개 행)]")
        print(merged_df.head())

        # 5. 병합된 파일을 CSV로 저장
        output_filename = 'merged_file.csv'
        # 한글 깨짐 방지를 위해 'utf-8-sig' 인코딩 사용
        merged_df.to_csv(output_filename, index=False, encoding='utf-8-sig')

        # 6. 저장된 파일 다운로드
        print(f"\n'{output_filename}' 파일이 생성되었습니다. 파일을 다운로드합니다.")
        files.download(output_filename)
    else:
        print("\n[오류] 유효한 파일 3개를 모두 처리하지 못했습니다. 파일 내용을 다시 확인해주세요.")

병합할 파일 3개를 업로드해주세요. (CSV 또는 XLSX)


Saving openalex_results (2019-01-01, 2019-12-31).csv to openalex_results (2019-01-01, 2019-12-31).csv
Saving openalex_results (2020-01-01, 2020-12-31).csv to openalex_results (2020-01-01, 2020-12-31).csv
Saving openalex_results (2021-01-01, 2021-12-31).csv to openalex_results (2021-01-01, 2021-12-31).csv
'openalex_results (2019-01-01, 2019-12-31).csv' 파일을 읽는 중...
'openalex_results (2020-01-01, 2020-12-31).csv' 파일을 읽는 중...
'openalex_results (2021-01-01, 2021-12-31).csv' 파일을 읽는 중...

✅ 파일 병합이 성공적으로 완료되었습니다.

[병합된 데이터 정보]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2806 entries, 0 to 2805
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        2806 non-null   object
 1   title     2744 non-null   object
 2   date      2806 non-null   object
 3   year      2806 non-null   int64 
 4   type      2806 non-null   object
 5   abstract  2466 non-null   object
dtypes: int64(1), object(5)
memory usage: 131.7+ KB

[병합된 데이터 미리보

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
from google.colab import files
import io

uploaded = files.upload()
uploaded = 'merged_file.csv'

Saving openalex_results_merged (2019-01-01, 2021-12-31).csv to openalex_results_merged (2019-01-01, 2021-12-31).csv


병합본에 대하여 전처리 - 1번, 결측값 제거

In [ ]:
# -*- coding: utf-8 -*-
"""
이미 병합된 CSV 파일을 불러와 결측값이 있는 행을 제거하는 스크립트입니다.
"""

import pandas as pd
from google.colab import files
import os

# 1. 처리할 파일 이름 지정
# 이전 단계에서 'merged_file.csv'로 저장했다면 그대로 두시면 됩니다.
# 만약 다른 이름으로 저장했다면 파일명을 수정해주세요.

input_filename = 'merged_file.csv'

# 2. 파일 불러오기 및 전처리
if not os.path.exists(input_filename):
    print(f"[오류] '{input_filename}' 파일을 찾을 수 없습니다.")
    print("Colab 왼쪽에 있는 파일 탐색기에서 파일이 존재하는지 확인해주세요.")
    print("파일이 없다면, 이전 병합 코드를 먼저 실행해야 합니다.")
else:
    try:
        print(f"'{input_filename}' 파일을 불러옵니다...")
        # CSV 파일을 DataFrame으로 읽기
        df = pd.read_csv(input_filename)

        print("\n 파일을 성공적으로 불러왔습니다.")
        print(f"처리 전 전체 행의 수: {len(df)}개")

        # 3. 전처리 전 결측값 확인
        print("\n[제거 전 칼럼별 결측값 개수]")
        missing_values = df.isnull().sum()
        print(missing_values)

        # 결측값이 하나도 없는 경우
        if missing_values.sum() == 0:
            print("\n🎉 모든 데이터에 결측값이 없습니다. 추가 작업 없이 종료합니다.")
        else:
            # 4. 결측값이 있는 행 제거
            print("\n⏳ 결측값이 있는 행을 제거합니다...")
            cleaned_df = df.dropna()

            # 5. 결과 확인
            print("\n 결측값 제거가 완료되었습니다.")
            print(f"제거 전 행의 수: {len(df)}개")
            print(f"제거 후 남은 행의 수: {len(cleaned_df)}개")
            print(f"제거된 행의 수: {len(df) - len(cleaned_df)}개")

            print("\n[전처리 완료된 데이터 미리보기 (상위 5개 행)]")
            print(cleaned_df.head())

            # 6. 전처리된 파일을 CSV로 저장
            output_filename = 'preprocessed_file.csv'
            # 한글 깨짐 방지를 위해 'utf-8-sig' 인코딩 사용
            cleaned_df.to_csv(output_filename, index=False, encoding='utf-8-sig')

            # 7. 저장된 파일 다운로드
            print(f"\n'{output_filename}' 파일이 생성되었습니다. 파일을 다운로드합니다.")
            files.download(output_filename)

    except Exception as e:
        print(f"파일을 처리하는 중 오류가 발생했습니다: {e}")

'merged_file.csv' 파일을 불러옵니다...

✅ 파일을 성공적으로 불러왔습니다.
처리 전 전체 행의 수: 2806개

[제거 전 칼럼별 결측값 개수]
id            0
title        62
date          0
year          0
type          0
abstract    340
dtype: int64

⏳ 결측값이 있는 행을 제거합니다...

✅ 결측값 제거가 완료되었습니다.
제거 전 행의 수: 2806개
제거 후 남은 행의 수: 2406개
제거된 행의 수: 400개

[전처리 완료된 데이터 미리보기 (상위 5개 행)]
                                 id  \
0  https://openalex.org/W2997687173   
1  https://openalex.org/W2998466307   
2  https://openalex.org/W3047301507   
3  https://openalex.org/W2997570731   
4  https://openalex.org/W2999667833   

                                               title        date  year  \
0  Groin Injuries in Soccer: Investigating the Ef...  2019-12-31  2019   
1  Effects of complex movements on the brain as a...  2019-12-31  2019   
2  The Body as a Machine in the Works of Soviet A...  2019-12-31  2019   
3  Motor speech and motor limb status in athletes...  2019-12-30  2019   
4  FUTBOLCULARDA 8 HAFTALIK STATİK GERME ANTRENMA...  2019-12-30  2019

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

영어 이외 언어로 작성된 행 제거

In [ ]:
# -*- coding: utf-8 -*-
"""
'preprocessed_file.csv' 파일을 불러와,
'초록' 칼럼의 언어를 감지하고 영어가 아닌 행을 모두 제거합니다.
"""

# 1. 언어 감지 라이브러리 설치
# 이 셀을 처음 실행할 때만 설치 과정이 표시됩니다.
print("언어 감지 라이브러리(langdetect)를 설치합니다...")
!pip install langdetect -q

import pandas as pd
from google.colab import files
from langdetect import detect, LangDetectException
import os
from tqdm.auto import tqdm

# Pandas의 progress_apply를 사용하기 위해 tqdm을 등록합니다.
tqdm.pandas()

# 2. 처리할 파일 이름과 칼럼 지정
input_filename = 'preprocessed_file.csv'
# --------------------------------------------------------------------------
# 중요: 초록 데이터가 들어있는 칼럼 이름을 정확하게 입력해주세요.
# 예: 'abstract', '요약', '초록' 등
abstract_column_name = 'abstract'
# --------------------------------------------------------------------------

# 3. 언어 감지 함수 정의
def detect_language(text):
    """텍스트의 언어를 감지하여 언어 코드를 반환하는 함수 (예: 'en', 'ko')"""
    try:
        # text가 유효한 문자열인지 확인
        if isinstance(text, str) and len(text.strip()) > 10: # 너무 짧은 텍스트는 감지 오류가 많아 제외
            return detect(text)
        else:
            return 'unknown'  # NaN, 빈 문자열, 또는 너무 짧은 텍스트
    except LangDetectException:
        # langdetect가 언어를 감지하지 못하는 경우 (예: 숫자, 기호만 있는 경우)
        return 'error'

# 4. 파일 불러오기 및 전처리
if not os.path.exists(input_filename):
    print(f"\n[오류] '{input_filename}' 파일을 찾을 수 없습니다.")
    print("Colab 왼쪽에 있는 파일 탐색기에서 파일이 존재하는지 확인해주세요.")
    print("파일이 없다면, 이전 전처리 코드를 먼저 실행해야 합니다.")
else:
    try:
        print(f"\n'{input_filename}' 파일을 불러옵니다...")
        df = pd.read_csv(input_filename)

        # 지정한 칼럼이 존재하는지 확인
        if abstract_column_name not in df.columns:
            print(f"\n[오류] '{abstract_column_name}' 칼럼을 찾을 수 없습니다.")
            print(f"현재 파일에 있는 칼럼: {df.columns.tolist()}")
            print("코드 상단의 'abstract_column_name' 변수를 실제 칼럼 이름으로 수정해주세요.")
        else:
            print(f"\n 파일을 성공적으로 불러왔습니다. (전체 행: {len(df)}개)")

            # 5. 각 행의 초록 언어 감지
            # 데이터가 많을 경우 시간이 걸릴 수 있어 진행률 바(progress bar)를 표시합니다.
            print(f"\n⏳ '{abstract_column_name}' 칼럼의 언어를 감지합니다. (시간이 소요될 수 있습니다)")
            # apply 대신 progress_apply를 사용하여 진행 상황 확인
            df['language'] = df[abstract_column_name].progress_apply(detect_language)

            # 6. 영어로 작성된 행만 필터링
            print("\n 언어 감지가 완료되었습니다. 영어로 된 초록만 필터링합니다...")
            english_df = df[df['language'] == 'en'].copy() # .copy()를 사용하여 경고 메시지 방지

            # 7. 결과 확인
            print(f"\n처리 전 행의 수: {len(df)}개")
            print(f"영어로 된 초록 행의 수: {len(english_df)}개")
            print(f"제거된 행의 수: {len(df) - len(english_df)}개")

            # 언어 감지에 사용된 'language' 칼럼은 최종 파일에서 제외
            english_df = english_df.drop(columns=['language'])

            print("\n[필터링 완료된 데이터 미리보기 (상위 5개 행)]")
            print(english_df.head())

            # 8. 필터링된 파일을 CSV로 저장
            output_filename = 'english_abstracts_only.csv'
            english_df.to_csv(output_filename, index=False, encoding='utf-8-sig')

            # 9. 저장된 파일 다운로드
            print(f"\n'{output_filename}' 파일이 생성되었습니다. 파일을 다운로드합니다.")
            files.download(output_filename)

    except Exception as e:
        print(f"파일을 처리하는 중 오류가 발생했습니다: {e}")

언어 감지 라이브러리(langdetect)를 설치합니다...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done

'preprocessed_file.csv' 파일을 불러옵니다...

✅ 파일을 성공적으로 불러왔습니다. (전체 행: 2406개)

⏳ 'abstract' 칼럼의 언어를 감지합니다. (시간이 소요될 수 있습니다)


  0%|          | 0/2406 [00:00<?, ?it/s]


✅ 언어 감지가 완료되었습니다. 영어로 된 초록만 필터링합니다...

처리 전 행의 수: 2406개
영어로 된 초록 행의 수: 2366개
제거된 행의 수: 40개

[필터링 완료된 데이터 미리보기 (상위 5개 행)]
                                 id  \
0  https://openalex.org/W2997687173   
1  https://openalex.org/W2998466307   
2  https://openalex.org/W3047301507   
3  https://openalex.org/W2997570731   
4  https://openalex.org/W2999667833   

                                               title        date  year  \
0  Groin Injuries in Soccer: Investigating the Ef...  2019-12-31  2019   
1  Effects of complex movements on the brain as a...  2019-12-31  2019   
2  The Body as a Machine in the Works of Soviet A...  2019-12-31  2019   
3  Motor speech and motor limb status in athletes...  2019-12-30  2019   
4  FUTBOLCULARDA 8 HAFTALIK STATİK GERME ANTRENMA...  2019-12-30  2019   

      type                                           abstract  
0  article  ABSTRACT Purpose The sudden rise in the injury...  
1  article  Non-linearity is considered to be an essential...  
2  art

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

이제 이 csv파일을 하나하나 읽어가며 실제 유효한 것을 분석하기 위한 xlsx파일변환

In [ ]:
# -*- coding: utf-8 -*-
"""
CSV 파일을 XLSX (엑셀) 파일로 변환하는 스크립트입니다.
엑셀 파일 생성을 위해 'openpyxl' 라이브러리를 설치합니다.
"""

# 1. 엑셀 파일 처리 라이브러리 설치
print("엑셀 파일 처리를 위한 라이브러리(openpyxl)를 설치합니다...")
!pip install openpyxl -q

import pandas as pd
from google.colab import files
import os

# 2. 변환할 파일과 저장할 파일 이름 지정
input_csv_file = 'english_abstracts_only.csv'
output_xlsx_file = 'english_abstracts_only.xlsx'

# 3. 파일 존재 여부 확인 및 변환 실행
if not os.path.exists(input_csv_file):
    print(f"\n[오류] 원본 파일인 '{input_csv_file}'을(를) 찾을 수 없습니다.")
    print("Colab 왼쪽에 있는 파일 탐색기에서 파일이 존재하는지 확인해주세요.")
    print("파일이 없다면, 이전 '영어 초록 필터링' 코드를 먼저 실행해야 합니다.")
else:
    try:
        # 4. CSV 파일 읽기
        print(f"\n'{input_csv_file}' 파일을 읽어옵니다...")
        df = pd.read_csv(input_csv_file)
        print(" CSV 파일을 성공적으로 불러왔습니다.")

        # 5. XLSX 파일로 저장
        # index=False 옵션은 엑셀 파일에 불필요한 인덱스 열이 추가되는 것을 방지합니다.
        print(f"\n'{output_xlsx_file}' 이름으로 엑셀 파일 변환을 시작합니다...")
        df.to_excel(output_xlsx_file, index=False, engine='openpyxl')

        # 6. 변환된 엑셀 파일 다운로드
        print(f" 엑셀 파일 변환이 완료되었습니다. 파일을 다운로드합니다.")
        files.download(output_xlsx_file)

    except Exception as e:
        print(f"\n파일을 변환하는 중 오류가 발생했습니다: {e}")

엑셀 파일 처리를 위한 라이브러리(openpyxl)를 설치합니다...

'english_abstracts_only.csv' 파일을 읽어옵니다...
✅ CSV 파일을 성공적으로 불러왔습니다.

'english_abstracts_only.xlsx' 이름으로 엑셀 파일 변환을 시작합니다...
✅ 엑셀 파일 변환이 완료되었습니다. 파일을 다운로드합니다.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

LLM 서빙 - 초록이 관련성이 있는지 검토

In [ ]:
!pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 2.1 MB/s eta 0:00:00


In [ ]:
# -*- coding: utf-8 -*-
"""
'english_abstracts_only.csv' 파일의 'abstract' 내용을 바탕으로,
LLM을 사용하여 각 뉴스가 축구 도메인의 인체공학과 관련이 있는지(1) 아닐지(0)를 판단하고,
원본의 모든 컬럼을 포함하여 결과를 저장합니다.
"""

import os
import pandas as pd
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage
from google.colab import drive
from tqdm.auto import tqdm # 진행상황 확인을 위해 tqdm 추가

# --------------------------------------------------------------------------
# ---  1. 설정 (사용자 확인 및 수정 필요) ---
# --------------------------------------------------------------------------

# OpenAI API 키를 여기에 입력하세요.
# 예: "sk-..."
os.environ["OPENAI_API_KEY"] = "Your API Key"

# 분석할 입력 파일 경로
input_filename = 'english_abstracts_only.csv'

# 입력 파일(english_abstracts_only.csv)의 실제 컬럼 이름을 영어로 매핑합니다.
title_column = 'title'
abstract_column = 'abstract'
date_column = 'date'

# 결과를 저장할 구글 드라이브 경로와 파일 이름
save_dir = "/content/drive/My Drive/텍스트마이닝"
save_filename = "텍스트마이닝_초록_검사결과_전체컬럼1921.csv" # 저장될 파일 이름
save_path = os.path.join(save_dir, save_filename)
save_path_xlsx = save_path.replace(".csv", ".xlsx")

# LLM 모델 설정
chat_model = ChatOpenAI(model="gpt-4o-mini")

# 한 번에 처리할 뉴스 기사 수 (API 요청 빈도 조절)
batch_size = 5

# 구글 드라이브 마운트
try:
    drive.mount('/content/drive')
except Exception as e:
    print(f"드라이브 마운트 중 오류 발생: {e}")

# --------------------------------------------------------------------------
# ---  2. 프롬프트 정의 (사용자 요청에 따라 원본 유지) ---
# --------------------------------------------------------------------------
prompt_header = """
We collected data from article abstracts to conduct an analysis of research trends in the field of ergonomics and biomechanics in the soccer and football domain from 2022 to 2024.

For the article abstracts, we searched with the keywords (Biomechanics and football) OR (Biomechanics and soccer) OR (Ergonomics and football) OR (Ergonomics and soccer), and each of these papers was checked to see if it was indeed in the field of
ergonomics and biomechanics in the domain of soccer and football. Review the titles and abstracts of the collected papers to determine if the
Determine if the papers are actually related to the field of ergonomics and biomechanics and if they are research in the soccer and football domain.


** Judging criteria: ** 1.
1. 1 Relevant → Research paper on ergonomics and biomechanics in the domain of soccer and football
2. 0 Unrelated → Papers not related to research in ergonomics and biomechanics in the soccer and football domain

Enter only numbers on a single line, separated by commas, in the following format (same as the news order)
Example: 1, 0, 0, 0, 1
"""

# --------------------------------------------------------------------------
# ---  3. 기존 저장 파일 불러오기 또는 새로 시작 ---
# --------------------------------------------------------------------------
os.makedirs(save_dir, exist_ok=True)

try:
    if os.path.exists(save_path) and os.path.getsize(save_path) > 0:
        df_saved = pd.read_csv(save_path)
        processed_titles = set(df_saved[title_column])
        print(f" 기존 결과 파일('{save_filename}')을 불러왔습니다. 총 {len(processed_titles)}개의 기사가 이미 처리되었습니다.")
    else:
        processed_titles = set()
        print(" 새로운 결과 파일을 생성합니다.")
except Exception as e:
    processed_titles = set()
    print(f" 기존 결과 파일 로딩 중 오류 발생: {e}. 새로 시작합니다.")


# --------------------------------------------------------------------------
# ---  4. 단일 파일 처리 실행 ---
# --------------------------------------------------------------------------
if not os.path.exists(input_filename):
    print(f" [오류] 입력 파일 '{input_filename}'을 찾을 수 없습니다. 파일이 현재 Colab 세션에 있는지 확인해주세요.")
else:
    print(f"\n 파일 처리 시작: '{input_filename}'")
    try:
        df_all = pd.read_csv(input_filename).reset_index(drop=True)

        df_to_process = df_all[~df_all[title_column].isin(processed_titles)].copy()

        if len(df_to_process) == 0:
            print(" 모든 기사가 이미 처리되었습니다. 추가 작업을 진행하지 않습니다.")
        else:
            print(f"총 {len(df_all)}개 기사 중 {len(processed_titles)}개는 이미 처리됨. 나머지 {len(df_to_process)}개를 처리합니다.")

            for i in tqdm(range(0, len(df_to_process), batch_size), desc="전체 진행률"):
                batch_df = df_to_process.iloc[i:i+batch_size]

                news_blocks = ""
                for _, row in batch_df.iterrows():
                    news_blocks += f"제목: {row[title_column]}\nabstract: {row[abstract_column]}\n날짜: {row[date_column]}\n\n"

                full_prompt = prompt_header + "\n" + news_blocks

                try:
                    response = chat_model.invoke([HumanMessage(content=full_prompt)])
                    raw_response = response.content.strip()
                    validity_list = [v.strip() for v in raw_response.split(',') if v.strip() in ['0', '1']]

                    if len(validity_list) != len(batch_df):
                        print(f" 경고: LLM 응답 개수({len(validity_list)})와 뉴스 개수({len(batch_df)})가 다릅니다. [응답: {raw_response}] 이 배치는 건너뜁니다.")
                        continue

                    results_to_save = []
                    for idx, v in enumerate(validity_list):
                        if v == "1":
                            # 1. 원본 행을 딕셔너리로 변환하여 모든 컬럼을 가져옵니다.
                            original_row_dict = batch_df.iloc[idx].to_dict()
                            # 2. 'validity' 컬럼을 추가합니다.
                            original_row_dict['validity'] = 1
                            # 3. 모든 정보가 담긴 딕셔너리를 저장 목록에 추가합니다.
                            results_to_save.append(original_row_dict)

                    if results_to_save:
                        write_header = not os.path.exists(save_path) or os.path.getsize(save_path) == 0
                        # DataFrame 생성 시 모든 컬럼이 자동으로 포함됩니다.
                        pd.DataFrame(results_to_save).to_csv(
                            save_path,
                            mode='a',
                            index=False,
                            header=write_header,
                            encoding='utf-8-sig'
                        )
                except Exception as llm_error:
                    print(f" LLM 호출 중 오류 발생 (배치 {i//batch_size + 1}): {llm_error}")
    except Exception as e:
        print(f" 파일 처리 중 오류 발생: {e}")

# --------------------------------------------------------------------------
# ---  5. 최종 엑셀 변환 및 저장 ---
# --------------------------------------------------------------------------
if os.path.exists(save_path) and os.path.getsize(save_path) > 0:
    print("\n 최종 CSV 파일을 Excel로 변환하여 저장합니다...")
    try:
        df_final = pd.read_csv(save_path)
        df_final.to_excel(save_path_xlsx, index=False)
        print(f" 전체 완료! 결과가 '{save_filename}'와 '{os.path.basename(save_path_xlsx)}'로 저장되었습니다.")
    except Exception as e:
        print(f" Excel 저장 중 오류 발생: {e}")
else:
    print("\n 유효성이 1로 판단된 데이터가 없어 저장할 파일이 없습니다.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ 기존 결과 파일('텍스트마이닝_초록_검사결과_전체컬럼1921.csv')을 불러왔습니다. 총 735개의 기사가 이미 처리되었습니다.

📂 파일 처리 시작: 'english_abstracts_only.csv'
총 2366개 기사 중 735개는 이미 처리됨. 나머지 1618개를 처리합니다.


전체 진행률:   0%|          | 0/324 [00:00<?, ?it/s]

⚠️ 경고: LLM 응답 개수(6)와 뉴스 개수(5)가 다릅니다. [응답: 0, 0, 0, 0, 0, 0] 이 배치는 건너뜁니다.
⚠️ 경고: LLM 응답 개수(27)와 뉴스 개수(5)가 다릅니다. [응답: 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1] 이 배치는 건너뜁니다.
⚠️ 경고: LLM 응답 개수(6)와 뉴스 개수(5)가 다릅니다. [응답: 0, 0, 0, 0, 0, 0] 이 배치는 건너뜁니다.
⚠️ 경고: LLM 응답 개수(4)와 뉴스 개수(5)가 다릅니다. [응답: 0, 0, 1, 0] 이 배치는 건너뜁니다.
⚠️ 경고: LLM 응답 개수(4)와 뉴스 개수(5)가 다릅니다. [응답: 0, 0, 0, 0] 이 배치는 건너뜁니다.
⚠️ 경고: LLM 응답 개수(4)와 뉴스 개수(5)가 다릅니다. [응답: 0, 0, 0, 0] 이 배치는 건너뜁니다.

📦 최종 CSV 파일을 Excel로 변환하여 저장합니다...
✅ 전체 완료! 결과가 '텍스트마이닝_초록_검사결과_전체컬럼1921.csv'와 '텍스트마이닝_초록_검사결과_전체컬럼1921.xlsx'로 저장되었습니다.


In [ ]:
files.download(save_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>